In [1]:
import pandas as pd
from itertools import combinations
import numpy as np
import os
from scipy import sparse
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer

from dask.distributed import Client, LocalCluster, performance_report
import dask.dataframe as dd

In [2]:
def len_combos(n):
    r = 2
    return np.math.factorial(n) / np.math.factorial(n-2) / np.math.factorial(2)

In [32]:
49_891_442_330 - 30275826000
19_615_616_330

19615616330

In [3]:
api_map = pd.read_csv('../data/out/all-apps/api_map.csv')
package_map = pd.read_csv('../data/out/all-apps/package_map.csv', index_col='uid')
api_map.shape, package_map.shape

((5160691, 2), (408180, 1))

In [45]:
package_map.loc['package251']

package    Lcom/google/android/gms/internal/ads
Name: package251, dtype: object

In [47]:
api_map = pd.read_csv('../data/out/all-apps/api_map.csv')
api_map[api_map.api.str.startswith('Lcom/google/android/gms/internal/ads')]

,api,uid
3168,Lcom/google/android/gms/internal/ads/zzane;->z...,api3168
3169,Lcom/google/android/gms/internal/ads/zzane;->z...,api3169
3252,Lcom/google/android/gms/internal/ads/zzkb;->zz...,api3252
3253,Lcom/google/android/gms/internal/ads/zzamu;->z...,api3253
3269,Lcom/google/android/gms/internal/ads/zzane;->e...,api3269
...,...,...
5157722,Lcom/google/android/gms/internal/ads/mt;->zzd(...,api5157722
5157723,Lcom/google/android/gms/internal/ads/hk;->zzpk...,api5157723
5157724,Lcom/google/android/gms/internal/ads/hw;->zza(...,api5157724
5157725,Lcom/google/android/gms/internal/ads/jn;->zzt(...,api5157725


In [5]:
api_map['pkg'] = api_map.api.str.replace('[$;].*', '')
api_map

<ipython-input-5-cfbde97e475e>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  api_map['pkg'] = api_map.api.str.replace('[$;].*', '')


,api,uid,pkg
0,Lcom/bumptech/glide/Priority;->values()[Lcom/b...,api0,Lcom/bumptech/glide/Priority
1,Lcom/bumptech/glide/Priority;->ordinal()I,api1,Lcom/bumptech/glide/Priority
2,Landroid/widget/ImageView$ScaleType;->values()...,api2,Landroid/widget/ImageView
3,Landroid/widget/ImageView$ScaleType;->ordinal()I,api3,Landroid/widget/ImageView
4,Lcom/bumptech/glide/request/target/ViewTarget;...,api4,Lcom/bumptech/glide/request/target/ViewTarget
...,...,...,...
5160686,Lb/a/a/a/a/d/b;->b(Ljava/lang/String;)V,api5160686,Lb/a/a/a/a/d/b
5160687,Lb/a/a/a/a/d/c;->a(I)Ljava/util/List;,api5160687,Lb/a/a/a/a/d/c
5160688,Lb/a/a/a/a/d/c;->d()V,api5160688,Lb/a/a/a/a/d/c
5160689,Lb/a/a/a/a/d/b;->a(Ljava/lang/String;)J,api5160689,Lb/a/a/a/a/d/b


In [5]:
pkgs = api_map.groupby('pkg').api.unique()
pkgs.size

475645

In [7]:
for pkg in pd.Series(pkgs.index).sample(50):
    print(pkg)

LX/2n6;
Ls/c/a/a/f;
Lo/ῑ;
Lcom/monsou/a20130521173532/More$2;
Lb/h/a/b/d/b/u;
Lcom/igexin/push/util/i;
[Lcom/adyen/threeds2/internal/ui/widget/ExpandableInfoTextView$a;
Lcom/coinbase/android/databinding/LayoutNotificationPreferencesBindingImpl;
Lo/ƅ;
LX/196;
[Lcom/google/android/gms/internal/ads/zzbo$zza$zza;
Lcom/facebook/stetho/inspector/protocol/module/Network$WebSocketClosedParams;
Lcom/google/android/gms/games/internal/api/zzbu;
Lcom/microsoft/intune/mam/client/app/offline/OfflineNotifyWipeActivity$2;
Lf/l/a/m$b;
Lcom/google/firebase/crashlytics/internal/MissingNativeComponent$b;
Landroid/support/b/b;
Lwob;
Lcom/aerserv/sdk/utils/MultiKey;
Lcom/evernote/provider/dbupgrade/MessageThreadParticipantsTableUpgrade;
Lcom/tune/ma/playlist/TunePlaylistManager$2;
Landroidx/versionedparcelable/c$a;
Landroid/support/design/internal/ForegroundLinearLayout;
Lcom/bumptech/glide/c0008/p001;
Lkotlinx/coroutines/flow/FlowKt__MergeKt$map$$inlined$unsafeTransform$2$2$1;
Lcom/amazon/device/ads/bl;
LX

In [2]:
pd.read_csv('../data/out/all-apps/app-data/com.microsoft.todos.csv', dtype=str)

,app,api,invoke_type,class,method,package,context
0,com.microsoft.todos,Ljava/lang/Long;->valueOf(J)Ljava/lang/Long;,static,Lb;,Lb;->a(J)I,Ljava/lang,"invoke-static {p0, p1}, Ljava/lang/Long;->valu..."
1,com.microsoft.todos,Ljava/lang/Long;->hashCode()I,virtual,Lb;,Lb;->a(J)I,Ljava/lang,"invoke-virtual {p0}, Ljava/lang/Long;->hashCod..."
2,com.microsoft.todos,Ljava/lang/Integer;->valueOf(I)Ljava/lang/Inte...,static,La;,La;->a(I)I,Ljava/lang,"invoke-static {p0}, Ljava/lang/Integer;->value..."
3,com.microsoft.todos,Ljava/lang/Integer;->hashCode()I,virtual,La;,La;->a(I)I,Ljava/lang,"invoke-virtual {p0}, Ljava/lang/Integer;->hash..."
4,com.microsoft.todos,Landroidx/recyclerview/widget/StaggeredGridLay...,direct,Landroidx/recyclerview/widget/StaggeredGridLay...,Landroidx/recyclerview/widget/StaggeredGridLay...,Landroidx/recyclerview/widget,"invoke-direct {v0}, Landroidx/recyclerview/wid..."
...,...,...,...,...,...,...,...
188138,com.microsoft.todos,Ljava/lang/StringBuilder;->append(I)Ljava/lang...,virtual,Lcom/google/android/gms/common/internal/w/b$a;,Lcom/google/android/gms/common/internal/w/b$a;...,Ljava/lang,"invoke-virtual {v2, v0}, Ljava/lang/StringBuil..."
188139,com.microsoft.todos,Ljava/lang/StringBuilder;->append(Ljava/lang/S...,virtual,Lcom/google/android/gms/common/internal/w/b$a;,Lcom/google/android/gms/common/internal/w/b$a;...,Ljava/lang,"invoke-virtual {v2, p1}, Ljava/lang/StringBuil..."
188140,com.microsoft.todos,Ljava/lang/StringBuilder;->append(I)Ljava/lang...,virtual,Lcom/google/android/gms/common/internal/w/b$a;,Lcom/google/android/gms/common/internal/w/b$a;...,Ljava/lang,"invoke-virtual {v2, p2}, Ljava/lang/StringBuil..."
188141,com.microsoft.todos,Ljava/lang/StringBuilder;->toString()Ljava/lan...,virtual,Lcom/google/android/gms/common/internal/w/b$a;,Lcom/google/android/gms/common/internal/w/b$a;...,Ljava/lang,"invoke-virtual {v2}, Ljava/lang/StringBuilder;..."


In [48]:
len_combos(301151)

90691623650.0

In [30]:
api_sets_total = pd.read_pickle('../data/temp/package_api_sets.pkl').apply(len).apply(len_combos).sum()
api_sets_total

49891442330.0

In [2]:
os.chdir('/home/rcgonzal/DSC180Malware/m2v-adversarial-hindroid/')
!pwd

/home/rcgonzal/DSC180Malware/m2v-adversarial-hindroid


In [3]:
outfolder = 'data/out/miniset-train'

In [10]:
from src.data.hindroid import *

%load_ext autoreload
%autoreload 2

In [11]:
%time make_models(outfolder)

Fitting AAT model...
Fitting ABAT model...
Fitting APAT model...
Fitting ABPBTAT model...
Fitting APBPTAT model...
              acc    recall        f1
kernel                               
AAT      0.957589  1.000000  0.957871
ABAT     0.939732  0.962963  0.939052
APAT     0.946429  0.990741  0.946903
ABPBTAT  0.944196  0.976852  0.944072
APBPTAT  0.904018  0.944444  0.904656
CPU times: user 50min 59s, sys: 4min 15s, total: 55min 15s
Wall time: 55min 15s


In [4]:
B_edges_filepath = os.path.join('data', 'temp', 'B_edges.csv')
P_edges_filepath = os.path.join('data', 'temp', 'P_edges.csv')
# pd.read_csv(P_edges_filepath).shape

In [4]:
pd.read_csv(os.path.join('data', 'out', 'miniset-train', 'api_map.csv')).shape

(2547149, 2)

In [5]:
pd.read_csv(os.path.join('data', 'out', 'miniset-train', 'package_map.csv')).shape

(246035, 2)

In [5]:
num_apis = 2547149

In [6]:
def parse_edges(chunk, num_apis):
        row = chunk.source
        col = chunk.target
        data = [1 for i in range(chunk.target.size)]
        return sparse.coo_matrix((data, (row, col)), shape=(num_apis, num_apis))

In [9]:
print("Constructing P matrix (Step 2)...")
# mlb = MultiLabelBinarizer(sparse_output=True)
# mlb.fit([(api,) for api in range(2547149)])
# P_mat = mlb.transform(pd.read_csv(P_edges_filepath).groupby('source').target.unique().compute())
# P_mat
print('loading data')
edges = pd.read_csv(P_edges_filepath)
print('making csr matrix')
P_mat = (
    sparse.csr_matrix(
        ([True for i in range(edges.shape[0])], (edges.source, edges.target)),
        shape=(num_apis, num_apis),
        dtype=bool
    )
)
P_mat.setdiag(1)
print("saving matrix")
sparse.save_npz(os.path.join(outfolder, 'hindroid', 'P_mat.npz'), P_mat.tocsr().astype(int))

Constructing P matrix (Step 2)...
loading data
making csr matrix


/home/rcgonzal/.local/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


saving matrix


In [10]:
import sys
sys.getsizeof(P_mat)

48

In [13]:
P_mat

<2547149x2547149 sparse matrix of type '<class 'numpy.bool_'>'
	with 1563269501 stored elements in Compressed Sparse Row format>

In [57]:
# app_list_path = os.path.join(outfolder, 'app_list.csv')
# nodes_path = os.path.join(outfolder, 'nodes.json')
# edge_path = os.path.join(outfolder, 'edges.csv')
# graph_path = os.path.join(outfolder, 'graph.pkl')
# feature_path = os.path.join(outfolder, 'features.csv')
# app_heap_path = os.path.join('data', 'out', 'all-apps', 'app-data/')
# metapath_walk_outpath = os.path.join(outfolder, 'metapath_walk.json')

# # generate app list
# apps_df = pd.read_csv(app_list_path)
# app_data_list = app_heap_path + apps_df.app + '.csv'

# with Client() as client, performance_report(os.path.join(outfolder, "performance_report.html")):
#     print(f"Dask Cluster: {client.cluster}")
#     print(f"Dashboard port: {client.scheduler_info()['services']['dashboard']}")

#     data = dd.read_csv(list(app_data_list), dtype=str)
#     pkg_info = data[['package', 'api']].groupby('package').package
#     pkg_info = pkg_info.value_counts().compute()

Dask Cluster: LocalCluster('tcp://127.0.0.1:33715', workers=4, threads=8, memory=68.72 GB)
Dashboard port: 8787


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

In [ ]:
pkg_info

In [18]:
pd.read_csv('../data/out/miniset-train/package_map.csv', index_col='uid').size

246035

In [42]:
pd.read_csv('../data/out/miniset-train/app_list.csv')

,app,app_dir,category
0,74b6098ac92ee92b54b5aefc8ecd5325,/teams/DSC180A_FA20_A00/a04malware/malware/Fak...,malware
1,34a34f03ebe76911b0f51c30bbc34b21,/teams/DSC180A_FA20_A00/a04malware/malware/Fak...,malware
2,373ed7aa5c726d29483a9d4aee692dbb,/teams/DSC180A_FA20_A00/a04malware/malware/Sim...,malware
3,com.gamevilusa.mlbpilive.android.google.global...,/teams/DSC180A_FA20_A00/a04malware/popular-app...,popular-apps
4,0f65a914c13a88fe561cf28d028919ac,/teams/DSC180A_FA20_A00/a04malware/malware/Fak...,malware
...,...,...,...
443,com.perblue.portalquest,/teams/DSC180A_FA20_A00/a04malware/popular-app...,popular-apps
444,80efa7feee3aa5da064db9f6b9092eb5,/teams/DSC180A_FA20_A00/a04malware/malware/Fak...,malware
445,7c5f4c915cb0017ec41d0aebfccfbbcf,/teams/DSC180A_FA20_A00/a04malware/malware/Sim...,malware
446,ec43a0cc9d22d738ecd9f21bf53e6756,/teams/DSC180A_FA20_A00/a04malware/malware/Gin...,malware


In [5]:
for i in combinations([i for i in range(30_000)], r=2):
    pass

In [12]:
np.math.factorial

<function math.factorial(x, /)>

In [15]:
n = 174000
r = 2

In [17]:
np.math.factorial(n) / np.math.factorial(n-r)

30275826000.0

In [ ]:
sparse.csr(shape=(num_apis, num_apis))

In [20]:
import sys

for i in range(1000, 100000, 1000):
    print(f'\r{i}: ', end='')
    print(f'{sys.getsizeof(list(combinations(range(i), r=2)))} btyes', end='')

17000: 1032132344 btyes2267512 btyes815511896 btyes917450928 btyes

KeyboardInterrupt: 

In [35]:
num_apis = pd.Series({
    'package1029': 12128,
    'package119': 14488,
    'package1269': 33490,
    'package1294': 25454,
    'package133': 10883,
    'package139': 17335,
    'package141621': 38315,
    'package227134': 27881,
    'package2276': 11418,
    'package2363': 106893,
    'package251': 174318,
    'package2816': 18863,
    'package286': 10675,
    'package338': 10943,
    'package580': 16923
})

In [36]:
pkg = pd.Series({'package1029': 'Lcom/google/protobuf',
'package119': 'Landroidx/appcompat/widget',
'package1269': 'Lcom/google/android/gms/internal/measurement',
'package1294': 'Lcom/google/android/gms/measurement/internal',
'package133': 'Landroidx/fragment/app',
'package139': 'Landroidx/recyclerview/widget',
'package141621': 'LX',
'package227134': 'Lcal',
'package2276': 'Lcom/vungle/publisher',
'package2363': 'Lcom/google/android/gms/internal',
'package251': 'Lcom/google/android/gms/internal/ads',
'package2816': 'Lo',
'package286': 'Lcom/google/android/gms/common/api/internal',
'package338': 'Lcom/google/android/gms/games/internal',
'package580': 'Landroid/support/v7/widget'})

In [40]:
pd.DataFrame().assign(pkg=pkg, num_apis=num_apis).sort_values('num_apis', ascending=False)

,pkg,num_apis
package251,Lcom/google/android/gms/internal/ads,174318
package2363,Lcom/google/android/gms/internal,106893
package141621,LX,38315
package1269,Lcom/google/android/gms/internal/measurement,33490
package227134,Lcal,27881
package1294,Lcom/google/android/gms/measurement/internal,25454
package2816,Lo,18863
package139,Landroidx/recyclerview/widget,17335
package580,Landroid/support/v7/widget,16923
package119,Landroidx/appcompat/widget,14488
